## Get Evaluations on all inference files and gather findings on quantitative metrics (such as _Cosine Similarity_) and subjective metrics on various criteria using _LLM as a judge_ - Max Voting & Average Pooling using PoLL (Panel of LLM Evaluators)

---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

#### This step of the solution focusses on getting evaluations on the quality of responses. It does so by gathering the following information and performing the steps below:

- **Gets all per inference request file**: This step first accesses and gets all of the per inference request files into a dataframe, containing the response from the LLM as well as the ground truth, if any is provided. 

- **Generates quantitative metrics for evaluation**: Calculate quantitative metrics to measure similarity and accuracy, for example _Cosine Similarity_. This helps in getting a quantitative overall score to the entire dataset in terms of which model generates outputs that are most similar and accurate to the ground truth (if any is provided). With this statistic, customers and users of the open source community can make business level judgements. 

- **Uses an _LLM as a judge_ approach to get subjective evaluations**: Refer to this [paper](https://arxiv.org/pdf/2404.18796). We use the following ways to evaluate the responses from the `candidate models` (models used to generate inferences)

    1. **Max Voting**: When a dataset provides a ground truth, we use a technique called `Max Voting`. Here, we use PoLL, or a panel of LLM evaluators, from different model families to evaluate each candidate model's response based on whether it generates a `correct` or an `incorrect` answer simply based on ground truth comparisons. Using models from different families as a PoLL, increases it's evaluation ability to be close to that of a human evaluation, and eliminates the intra model bias.
    
    2. **Average Pooling**: When a dataset does not provide a ground truth, or if a task being evaluated needs to be given subjective level judgements, that is when we use `Average Pooling`. We use a specific subjective level criteria and then evaluate the candidate model responses on a scale of 1-5 for each PoLL. Using this, we get an average score and then can evaluate how each candidate model was scored based on the PoLL evaluations.
    
FMBench uses this approach of PoLL to eradicate intra model bias by using models as judges from different model families. This brings the evaluation results closer to that of a human evaluation, makes the evaluation process more streamlined, consistent across all the responses, and reduces the latency and cost of evaluating the candidate models over time.
    
***All evaluations are generated in a JSON format for further downstream analytics on the evaluation results***

#### Import all of the necessary libraries below to run this notebook

In [ ]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import io
import ray
import time
import json
import glob
import pandas as pd
from numpy import dot
from fuzzywuzzy import fuzz
from numpy.linalg import norm
from litellm import completion
from difflib import SequenceMatcher as SM
from sentence_transformers import SentenceTransformer

# Import seaborn and other related libraries for visualizations and plotting charts
import seaborn as sns
from pathlib import Path
from fmbench.utils import *
from fmbench.globals import *
from typing import List, Optional, Dict
import importlib.resources as pkg_resources
from fmbench import __version__ as fmbench_version

In [ ]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# initialize the ray service to run async calls in parallel to bedrock easily
if ray.is_initialized():
    ray.shutdown()
ray.init()

Load the Config.yml file contains information that is used across this benchmarking environment, such as information about the aws account, prompts, payloads to be used for invocations

In [ ]:
logger.info(f"CONFIG_FILE={CONFIG_FILE}")
config = load_main_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

#### Load the associated pricing config file

In [ ]:
# represents getting the config file from the s3 bucket/https path for pricing yml information
pricing_file_path: str = config['pricing'] 

# initialize the pricing config file to None
pricing_config: Optional[Dict] = None

# get the current config dir path
config_dir = Path(pkg_resources.files('fmbench'), 'configs')
logger.info(f"Using fmbench.configs directory: {config_dir}")

pricing_module = Path(config['pricing'])
logger.info(f"pricing config provided for inference from this model is --> {pricing_module}")
pricing_file_path = os.path.join(config_dir, pricing_module)
logger.info(f"pricing config file path is --> {pricing_file_path}")

pricing_config = load_config(pricing_file_path)
logger.info(f"pricing config file recorded: {json.dumps(pricing_config, indent=2)}")

In [ ]:
debug = False
if debug is True:
    metrics_path_file: str = os.path.join("..", "..", METADATA_DIR, METRICS_PATH_FNAME)
else:
    metrics_path_file: str = os.path.join(METADATA_DIR, METRICS_PATH_FNAME)
logger.info(f"cwd={os.getcwd()}, METADATA_DIR={METADATA_DIR}, METRICS_PATH_FNAME={METRICS_PATH_FNAME}, metrics_path_file={metrics_path_file}")
METRICS_DIR: str = Path(metrics_path_file).read_text().strip()
logger.info(f"metrics_path_file={metrics_path_file}, METRICS_DIR={METRICS_DIR}")

In [ ]:
file_path = os.path.join(METRICS_DIR, config["report"]["per_inference_request_file"])
# file_path='fmbench-bedrock-fmbench-1-us-east-1-role/data/metrics/yyyy=2024/mm=07/dd=17/hh=02/mm=43/per_inference_request_results.csv'
logger.info(f"File path containing the metrics per inference folder --> {file_path}")

# Read the file from S3
try:
    file_content = get_s3_object(config['aws']['bucket'], file_path)
    # Use pandas to read the CSV content
    df_per_inference = pd.read_csv(io.StringIO(file_content))
    logger.info(f"{file_path} read into dataframe of shape {df_per_inference.shape}, "
                f"cols={df_per_inference.columns}")
    logger.info(f"{file_path} contains results for the following endpoints={df_per_inference.endpoint_name.unique()}")
    logger.info(df_per_inference.head())
except Exception as e:
    logger.error(f"Error reading from S3: {e}")


In [ ]:
df_per_inference.head()

### Relationship between prompt token length and inference latency for different instances and concurrency levels

In [ ]:
df_per_inference.latency.describe()

### Use the `sentence-transformers/all-mpnet-base-v2` embeddings model to calculate the _Cosine Similarity_ scores 
---

This portion of the evaluation step does as follows:

1. Uses the `sentence-transformers/all-mpnet-base-v2` model from Hugging Face. This is a sentence-transformers model. It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

1. Use the embeddings model to get quantitative metrics from the inferences. This helps to get a similarity score between the ground truth answers from a dataset if any are given and the actual responses from the model received during inference.

1. If no ground truth is provided, cosine similarity is calculated between the response and the content provided to answer the question.embeddings_model_info

In [ ]:
# get the quantitiative evaluation information from the config file, such as the embeddings model
# to be used
embeddings_model_quantitative_info: Dict = config['model_evaluations']['quantitative_eval_info']


def load_model():
    """
    This function loads the sentence-transformers model based on the provided model ID.
    """
    try: 
        model=None
        model_id = embeddings_model_quantitative_info['embeddings_model_id'].get('model_id', None)
        if model_id:
            model = SentenceTransformer(model_id)
        else:
            raise ValueError("Model ID is not provided or invalid in the configuration.")
    except Exception as e:
        logger.error(f"The SentenceTransformer embeddings model could not be loaded: {e}")
        model=None
    return model

In [ ]:
# load the embeddings model to calculate the cosine similarity scores
model = load_model()


def calculate_cosine_similarity(text1: str, text2: str) -> float:
    """
    This function calculates the cosine similarity between two texts. In this case, 
    the cosine similarity is the comparison between the ground truth in the given dataset
    and the candidate model's response
    """
    try:
        cosine: float = None
        # returns the embedding for a given text using the sentence-transformers model.
        A = model.encode([text1])[0]
        B = model.encode([text2])[0]
        cosine = dot(A, B) / (norm(A) * norm(B))
    except Exception as e:
        logger.error(f"Cosine similarity was not calculated at this iteration: {e}")
        cosine=None
    return cosine

In [ ]:
logger.info(f"~Creating embeddings of all candidate model responses now. This might take a couple minutes~")

# Assuming df_per_inference is your DataFrame
df_per_inference['cosine_similarity_score'] = df_per_inference.apply(
    lambda row: calculate_cosine_similarity(row['completion'], row['ground_truth']), axis=1
)
df_per_inference.head()

### Lexical Match & Cosine Similarity Score Accuracy Evaluation Filter
---

Before having the Panel of LLM Evaluators evaluate each candidate model response, we pass those responses through a filtering step. In this step we use a threshold for a `Lexical match` and a `Cosine Similarity` score to define whether that answer is correct without having an LLM evaluate it. The thresholds for correctness is defined in the configuration files. 

The reason to do this is to make the evaluation process more like a hierarchy of checks, to make sure each and every candidate model response is evaluated appropriately. Additionally, filter steps to check for these scores to determine whether a candidate model reponse is correct, will narrow down the evaluation checks for the PoLL reducing the time and cost to complete all evaluations. This is specific to the `Ground Truth based approach`. 

For the lexical match, we use the `fuzzy` match approach `token_set_ratio` library to determine what percent of the two texts are similar.

**Note**: `Token_set_ratio` algorithm tokenizes both input strings, removes duplicate tokens, and calculates the similarity score based on the intersection and union of the token sets. It captures the essence of the strings’ content rather than their specific order.

In [ ]:
def calculate_token_set_ratio(text1: str, text2: str) -> float:
    """
    This function calculates the partial token match or fuzz ratio between two strings.
    If the fuzz ratio exceeds the threshold and the cosine similarity matches or exceeds the threshold, 
    then the answer is correct and it is not evaluated using a judge. If it is not, then it
    is parsed through the PoLL process
    """
    try:
        token_set_ratio: float = None
        if text1 and text2:
            token_set_ratio = fuzz.token_set_ratio(text1, text2) / 100.0
        else:
            token_set_ratio=None
    except Exception as e:
        logger.error(f"Error in calculating token set ratio: {e}")
        token_set_ratio=None
    return token_set_ratio

### Levenshtein distance algorithm
---
In information theory, linguistics, and computer science, the Levenshtein distance is a string metric for measuring the difference between two sequences. The Levenshtein distance between two words is the minimum number of single-character edits (insertions, deletions or substitutions) required to change one word into the other.

In [ ]:
def levenshtein_distance(s, t):
	m, n = len(s), len(t)
	if m < n:
		s, t = t, s
		m, n = n, m
	d = [list(range(n + 1))] + [[i] + [0] * n for i in range(1, m + 1)]
	for j in range(1, n + 1):
		for i in range(1, m + 1):
			if s[i - 1] == t[j - 1]:
				d[i][j] = d[i - 1][j - 1]
			else:
				d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1
	return d[m][n]


def calculate_levenshtein_distance(input_string, reference_string):
    distance = levenshtein_distance(input_string, reference_string)
    max_length = max(len(input_string), len(reference_string))
    similarity = 1 - (distance / max_length)
    return similarity

In [ ]:
# # These are examples from the LongBench dataset for testing purposes
# candidate_model_response: str = "Both Sinofranchetia and Stauntonia are from the Lardizabalaceae family. This information is mentioned in the passages for both genera."
# ground_truth: str = "a genus of flowering plant in the Lardizabalaceae family"
# ratio = calculate_levenshtein_distance(candidate_model_response, ground_truth)
# print(f"ratio calculated: {ratio}")

In [ ]:
# Compute the token set ratio for each row and add it as a new column
df_per_inference['token_set_ratio_value'] = df_per_inference.apply(
    lambda row: calculate_token_set_ratio(row['completion'], row['ground_truth']), axis=1
)

df_per_inference['levenshtein_distance'] = df_per_inference.apply(
    lambda row: calculate_levenshtein_distance(row['completion'], row['ground_truth']), axis=1
)

df_per_inference.head()

In [ ]:
# define the all_metrics path to send the evaluation metrics to
all_metrics_fpath = os.path.join(METRICS_DIR, config["report"]["all_metrics_file"])
csv_buffer = io.StringIO()
df_per_inference.to_csv(csv_buffer, index=False)
df_per_inference_with_cosine_similarity_scores_csv = csv_buffer.getvalue()
inference_cosine_similarity_scores_s3_path = os.path.join(METRICS_DIR, PER_INFERENCE_FILE_WITH_COSINE_SIMILARITY_SCORES)  # Define full S3 path

# Write the CSV data to S3
write_to_s3(df_per_inference_with_cosine_similarity_scores_csv, BUCKET_NAME, "", 
            METRICS_DIR, PER_INFERENCE_FILE_WITH_COSINE_SIMILARITY_SCORES)
logger.info(f"Per inference cosine similarity scores saved to s3://{BUCKET_NAME}/{inference_cosine_similarity_scores_s3_path}")

df_per_inference.head()

### Use _Panel of LLM Evaluators_ to get Subjective Evaluations on various evaluation criteria
---

In this portion of the notebook, we run evaluations on the content generated by different candidate models. We use two main evaluation methods: `Max Voting` and `Average Pooling`. To eliminate intra-model bias, we address this by scoring answer correctness based not on a single judge, but instead on a panel composed of multiple evaluator models. Similar pooling techniques are used to reduce variance in human annotations by normalizing out both natural variation in human judgements caused by their own subjective biases as well as human error. We use the following two techniques:

1. **Max Voting**: We use the PoLL to evaluate candidate model responses by checking its correctness compared to a provided ground truth answer in the dataset. We prompt each PoLL to evaluate and give the response in a JSON structure, giving a verdict on whether the response is correct or incorrect, and an explanation as to why that is. Using this, we can perform downstream analytics such as: 

    1. Calculate the overall accuracy of each model using the correct versus the (correct + incorrect) responses
    
    1. Calculate the `error rate` or frequency or incorrect responses
    
    1. Categorize the errors based on the explanations provided by the evaluators. Common categories might include misunderstanding the question, incomplete answers, factual inaccuracies
    
    1. Summary of overall correct/incorrect, and the best model based on the PoLL. Rank the models on Correctness versus Incorrectness.

1. **Average Pooling**: We use the PoLL to rate the response of each candidate model on a more subjective criteria. Here, we have the candidate model responses rated on a scale of 1-5 based on the subjective criteria and then get an explanation to that. Using this we can do as follows:

    1. Calculate the average score for each model across all questions to get an overall performance measure.
    
    1. Compute the standard deviation of the scores to understand the consistency of the model's performance.

1. Towards the end of all evaluations, a final layer of evaluation is added at the end. This layer utilizes another LLM that acts as a final summarizer. It takes in the ratings, answers generated from each unique model that was used in inference, to give a list of trends, overall patterns and observations as to which model is suited for a given task for a given dataset.

In [ ]:
# get the qualitative/subjective evaluation information from the config file to evaluate answers from different
# endpoints on various criteria
model_eval_subjective_info: Dict = config['model_evaluations']['subjective_eval_info']
eval_criteria_list = model_eval_subjective_info.get('eval_criteria', None)
logger.info(f"available llm as a judge evaluation information to use: {json.dumps(model_eval_subjective_info, indent=2)}")

In [ ]:
# get the inference parameters that the LLM judge panel will use while evaluating model candidate responses
INFERENCE_PARAMETERS_LLM_PANEL: Dict = config['model_evaluations']['subjective_eval_info'].get('inference_parameters', None)

In [ ]:
def get_panel_of_llm_evaluation(model_id: str,
                                  prompt: str):
    """
    Get inference using LiteLLM. This get's inference on the answers provided and evaluates each
    answer based on a given evaluation prompt template and the specific set of rules for each
    evaluation criteria.
    """
    # represents the service name
    print(f"get_inference, model_id={model_id}")
    service_name: str = "bedrock"
    # represents creating the bedrock model to invoke the litellm api for response for titan, llama and claude
    bedrock_model: str = f"{service_name}/{model_id}"
    # represents the current aws region
    aws_region = boto3.Session().region_name 
    # initialize the response dict
    ret = dict(exception=None,
               prompt=prompt,
               completion=None,
               completion_token_count=None,
               prompt_token_count=None,
               model_id=model_id)
    body = ret['prompt']
    os.environ["AWS_REGION_NAME"] = aws_region
    try:
        # Represents calling the litellm completion/messaging api utilizing the completion/embeddings API
        print(f"Invoking {bedrock_model}......")
        response = completion(model=bedrock_model,
                              messages=[{"content": body,"role": "user"}],
                              temperature=INFERENCE_PARAMETERS_LLM_PANEL.get('temperature', 0.1),
                              max_tokens=INFERENCE_PARAMETERS_LLM_PANEL.get('max_tokens', 100),
                              caching=INFERENCE_PARAMETERS_LLM_PANEL.get('caching', False))
        print(f"response: {response}")
        # iterate through the entire model response
        for idx, choice in enumerate(response.choices):
            # extract the message and the message's content from litellm
            if choice.message and choice.message.content:
                # extract the response from the dict
                ret["completion"] = choice.message.content.strip()
        # Extract number of input and completion prompt tokens        
        ret['prompt_token_count'] = response.usage.prompt_tokens
        ret['completion_token_count'] = response.usage.completion_tokens
    except Exception as e:
        logger.error(f"Exception occurred during invoking {model_id}, exception={e}")
        ret['exception'] = e
    logger.info(f"completion: {ret['completion']}")
    return ret

In [ ]:
def safe_filename(s):
    """
    convert a string to another string that can be used as a filename
    i.e. remove white space and non-word chars
    """
    if s is None:
        return "None"
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = re.sub(r"\s+", '-', s)
    return s

In [ ]:
def parse_as_json(x: str) -> Optional[Dict]:
    """
    Convert a string into a dictionary. Remove any
    stray whitespaces which could break the json parsing
    """
    d: Optional[Dict] = None
    try:
        x = x.replace("\n", "").replace("\t", "")
        d = json.loads(x)
    except Exception as e:
        print(f"parse_as_json, error parsing string as json, string={x}")
    return d

### Read the latest dataframe and run LLM as a judge evaluations on it

In [ ]:
df_per_inference.head()

### Prepare the evaluation prompt payloads
---

Here, the evaluation prompt template is used by the LLM judge to evaluate the answers on different criteria.
This prompt template function uses a set of rules, prompt template, the answer, and ground truth (if any) in the
evaluation solution

In [ ]:
def prepare_eval_prompts(eval_template: str,
                         answer: str, 
                         rules: str, 
                         context: str, 
                         ground_truth: Optional[str]):
    """
    This function prepares the evaluation prompts by preparing the standard eval prompt template
    with the rules of a given subjective criteria, context, answer and ground truth (if any ground truth is provided)
    """
    try:
        processed_eval_template: Optional[str] = None
        processed_eval_template = eval_template.format(
            rules=rules,
            answer=answer,
            context=context,
            ground_truth=ground_truth)
    except Exception as e:
        logger.error(f"Error encountered while generating the evaluation prompt template: {e}")
        processed_eval_template=None
    return processed_eval_template

In [ ]:
def clear_dir(dir_path: str):
    files = glob.glob(os.path.join(dir_path, "*"))
    for f in files:
        os.remove(f)

# create the metrics directory that stores all of the json files containing evaluations from all Panel of LLM evaluators
METRICS_PER_POLL_EVAL_DIR: str = os.path.join(METRICS_DIR, METRICS_PER_POLL_EVAL_DIR_NAME)
_ = list(map(clear_dir, [METRICS_PER_POLL_EVAL_DIR]))

In [ ]:
def run_llm_evals(i: int, total: int, row: Dict,  model_id: str, eval_method_name: str, uuid: str) -> Dict:
    """
    Runs the evaluation for one row 
    The eval prompt is already available in the row dictionary
    and we simply want to run the inference against the judge model.
    The results are returned in a new dictionary that contains the model 
    response and some fields from the original dictionary
    """
    try: 
        # save all the responses from the model in a dictionary
        resp: Dict = {}
        print(f"run_eval, row {i}/{total}, judge_model_id={model_id}, candidate model={row['endpoint_name']}")
        candidate_model_response: str = row['completion']
        # create the payload for model inference
        prompt = row[f'{model_id}_{method_name}_eval_prompt']
        # generate the evaluation on the data using the model judge
        resp = get_panel_of_llm_evaluation(model_id, prompt)
        # assign the completion from the candidate model to the `candidate_model_response`, 
        # and the actual evaluation will be contained in a field called `completion`
        resp['candidate_model_response'] = candidate_model_response
        logger.info(f"Panel of LLM evaluator {model_id} completion: {resp['completion']}")
        resp['candidate_model'] = row['endpoint_name']
        resp['cosine_similarity_score'] = row['cosine_similarity_score']
        resp['levenshtein_distance'] = row['levenshtein_distance']
        resp['token_set_ratio_value'] = row['token_set_ratio_value']
        # if there is a ground truth, include that in the json response
        if 'ground_truth' in row:
            resp['ground_truth'] = row['ground_truth']
    except Exception as e:
        logger.error(f"Error encountered while running evaluation: {e}")
        resp=None
    return resp

# we use Ray to parallize
@ray.remote
def async_run_eval(i: int, total: int, row: Dict, model_id: str, eval_method_name: str, uuid: str) -> Dict:
    print(f"async_run_eval, i={i}, total={total}, judge_model_info={model_id}, eval_method: {eval_method_name}, uuid: {uuid}")
    return run_llm_evals(i, total, row, model_id, eval_method_name, uuid)

In [ ]:
# convert the dataframe into a list of dicts as that is easy to parallize via Ray
df_per_inference_list = json.loads(df_per_inference.to_json(orient='records'))
logger.info(f"eval_records_list has {len(df_per_inference_list)} entries")

### Prepare evaluation prompt templates
---

This portion of the step prepares the evaluation prompt templates that are used in the evaluation process of using `Max Voting` or `Average Pooling` using the PoLL.

In [ ]:
model_eval_subjective_info

In [ ]:
# get the method that is being used to evaluate the content (which is either 
# max voting or average pooling)
method_name: str = config['model_evaluations']['PoLL_Composition_and_Voting'].get('method', None)
logger.info(f"The evaluation method FMBench is going to use to evaluate different model responses: {method_name}")
logger.info(f"judge panel being used to evaluate model responses: {model_eval_subjective_info.get('judge_panel_list', None)}")

In [ ]:
# Assuming fmbench is a valid Python package and scripts is a subdirectory within it
eval_prompts_dir = Path(pkg_resources.files('fmbench'), f"{config['s3_read_data']['prompt_template_dir']}/{config['s3_read_data']['eval_prompts_dir']}")
# Iterate through each LLM as a judge and each evaluation criterion
for llm_info in model_eval_subjective_info.get('judge_panel_list', None):
    model_id = llm_info['model_id']
    eval_prompt_template_fname: str = f"{llm_info.get('eval_prompt_template_name_max_voting', None)}.txt"
    eval_prompt_template_dir = llm_info.get('eval_prompt_template_dir', None)
    eval_prompt_template_path: str = os.path.join(eval_prompts_dir, eval_prompt_template_dir, eval_prompt_template_fname)
    logger.info(f"evaluation prompt template file path being used for {model_id}: {eval_prompt_template_path}")
    logger.info(f"evaluation prompt template file name: {eval_prompt_template_fname}")
    try:
        eval_prompt_template = Path(eval_prompt_template_path).read_text()
    except FileNotFoundError:
        logger.error(f"File not found: {eval_prompt_template_path}")
        continue

    print(f"Evaluation prompt template being used: {eval_prompt_template}")
    eval_instructions_fname: str = next((rule for rule in config['s3_read_data']['eval_instructions_files'] if method_name in rule), None)
    rules = Path(os.path.join(eval_prompts_dir, eval_instructions_fname)).read_text()
    logger.info(f"rules: {rules}")
    column_name = f"{model_id}_{method_name}_eval_prompt"
    logger.info(f"column_name: {column_name}")
    # prepare the prompt payloads with the existing candidate model responses, ground truth (if any), 
    # prompt template for the given criteria and rules to be used
    df_per_inference[column_name] = df_per_inference.apply(
        lambda r: prepare_eval_prompts(
            eval_prompt_template,
            r['completion'],
            rules,
            r['prompt'],
            r['ground_truth']
        ),
        axis=1
    )

In [ ]:
csv_buffer = io.StringIO()
df_per_inference.to_csv(csv_buffer, index=False)
df_per_inference_with_eval_prompt_payloads = csv_buffer.getvalue()
eval_prompt_payloads_for_inference = os.path.join(METRICS_DIR, PROCESSED_EVAL_PROMPT_PAYLOADS)  # Define full S3 path

# Write the CSV data to S3
write_to_s3(df_per_inference_with_eval_prompt_payloads, BUCKET_NAME, "", 
            METRICS_DIR, PROCESSED_EVAL_PROMPT_PAYLOADS)
logger.info(f"Per inference cosine similarity scores saved to s3://{BUCKET_NAME}/{eval_prompt_payloads_for_inference}")
df_per_inference.head()

In [ ]:
# convert the dataframe into a list of dicts as that is easy to parallize via Ray
eval_records_list = json.loads(df_per_inference.to_json(orient='records'))
logger.info(f"eval_records_list has {len(eval_records_list)} entries")

### Run LLM as a Judge Evaluations
---

In this portion of the step, FMBench performs the following actions:

1. If the method of evaluation is `Max Voting`, then in that case we suppose that a ground truth to the question from the context or task is pre existing in the dataset. We use the LLM panel of judges (in this case 3 judges), to give a verdict on whether the `answer` from the candidate models during inference is `correct` or `incorrect`. If the response is correct, then it gives it a `correct` and if not, then `incorrect`.

1. If the method of evaluation is `Average Pooling`, then in that case we suppose that the completion from the candidate models are supposed to be evlauated on a more subjective criteria rather than just deciding whether it is correct or incorrect compared to the ground truth. In this case, the average pooling prompt templates are used by the Judge Panel to give a rating out of 1-5 to each model completion on different criteria, such as relevancy, helpfulness, correctness, and so on.

1. Each model response is given in a JSON structure which is further used for downstream analytics, to decide the comparision of evaluation results between different model candidates and more.

***This step takes about ~6 minutes to complete. Model completion time depends on the PoLL models being used. `Llama3-70b`, `Cohere command-r-v1` and `claude 3 haiku` were used for this example***

In [ ]:
# get the llm as a judge panel list
judge_panel_list = model_eval_subjective_info.get('judge_panel_list', None)
logger.info(f"The judge panel list contains {len(judge_panel_list)} judges: {judge_panel_list}")

In [ ]:
logger.info(f"~Panel of LLM evaluators are going to start evaluating responses. This might take a couple of minutes~")

In [ ]:
def process_record_for_lexical_similarity(record):
    """
    Given a record, this function calculates the average of token set ratio and Levenshtein distance,
    and checks the cosine similarity. If the cosine similarity meets or exceeds the specified threshold,
    or if the average of token set ratio and Levenshtein distance meets or exceeds the specified threshold,
    the completion is correct and an explanation is given without going through an LLM evaluator
    """
    try:
        token_set_ratio = record['token_set_ratio_value']
        levenshtein_ratio = record['levenshtein_distance']
        cosine_similarity_score = record['cosine_similarity_score']

        is_correct = (cosine_similarity_score >= config['report']['cosine_similarity_threshold'] or 
                      token_set_ratio >= config['report']['token_set_ratio_threshold'] or
                      levenshtein_ratio >= config['report']['levenshtein_distance_threshold'])

        if is_correct:
            verdict = "correct"
            explanation = (
                f"Lexical match check passed with: "
                f"Token set ratio = {token_set_ratio * 100}%, "
                f"Levenshtein similarity match = {levenshtein_ratio * 100}%, "
                f"Cosine similarity = {cosine_similarity_score:.3f}, not going through a panel of LLM evaluator."
            )
            record.update({
                'candidate_model': record['endpoint_name'],
                'candidate_model_response': record['completion'],
                'completion': f'{{\n  "verdict": "{verdict}",\n  "explanation": "{explanation}"\n}}',
            })
        else:
            is_correct = False
    except Exception as e:
        logger.error(f"Error occurred while checking for text similarity: {e}")
        record = None
        is_correct = False
    return record, is_correct

In [ ]:
n = model_eval_subjective_info.get('run_parallel_inference_count', 5)
list_of_lists = [eval_records_list[i * n:(i + 1) * n] for i in range((len(eval_records_list) + n - 1) // n)]
resp_list = []
erroneous_count = 0
st = time.perf_counter()

# Iterate over the judge panel and sublists
for judge_panelist_info in judge_panel_list:
    logger.info(f"============Running inference for judge panelist {judge_panelist_info['model_id']} for {method_name} ============")
    for idx, sublist in enumerate(list_of_lists):
        model_id = judge_panelist_info['model_id']
        logger.info(f"Getting inference for list {idx + 1}/{len(list_of_lists)}, size of list={len(sublist)}")
        non_matching_records = []

        for record in sublist:
            # First, check if the current content of the record matches the threshold for 
            # token set ratio or cosine similarity
            processed_record, is_correct = process_record_for_lexical_similarity(record)
            if is_correct:
                resp_list.append(processed_record)
            else:
                non_matching_records.append(record)
        try:
            if non_matching_records:
                # Run inference in parallel for non-matching records
                resp_list.extend(ray.get([async_run_eval.remote(i + 1, len(non_matching_records), record, model_id, method_name, record['uuid'])
                                   for i, record in enumerate(non_matching_records)]))
        except Exception as e:
            logger.error(f"Error processing list {idx + 1}/{len(list_of_lists)}: {e}")
            erroneous_count += 1

    # Sleep for two seconds before moving on to the next model
    logger.info(f"~Sleeping for one second before the next Panel of LLM evaluates the responses~")
    time.sleep(1)

elapsed_time = time.perf_counter() - st
logger.info(f"Total elapsed time for inference: {elapsed_time:.2f} seconds")
logger.info(f"Total erroneous lists: {erroneous_count}")


### Send all Panel of LLM evaluator responses to S3 as `JSON` files

In [ ]:
def sanitize_filename(text):
    return "".join([c if c.isalnum() else "_" for c in text])


# Collect all of the panel of LLM evals and send them all as JSON files
# to s3
if resp_list:
    save_s3_list = []

    try:
        for resp in resp_list:
            llm_eval_response = json.dumps(resp, indent=2)
            candidate_model_id = resp.get('candidate_model', None)
            # Extract a few words from the poll eval response to append to the file name
            response_excerpt = " ".join(resp.get('candidate_model_response', "").split()[:5])
            sanitized_response_excerpt = sanitize_filename(response_excerpt)
            timestamp = time.time()
            llm_eval_json_fname = f"{candidate_model_id}_{timestamp}_{sanitized_response_excerpt}.json"
            response_s3_path = os.path.join(METRICS_PER_POLL_EVAL_DIR, llm_eval_json_fname)
            logger.info(f"sending llm eval response to s3 path prefix: {response_s3_path}")
            save_s3_list.append((llm_eval_response,
                                config['aws']['bucket'],
                                "",
                                METRICS_PER_POLL_EVAL_DIR,
                                llm_eval_json_fname))
        write_multiple_to_s3(save_s3_list)
    except Exception as e:
        logger.error(f"Error processing or writing to S3: {e}")
else:
    logger.info("No responses to write to S3")

### Perform downstream analytical tasks on each PoLL evaluation result
---

In [ ]:
# convert the results list into a dataframe for easy analytics
df_eval_results = pd.DataFrame(resp_list)
logger.info(f"df_eval_results shape={df_eval_results.shape}")
df_eval_results.dropna(axis=1, how='all')
df_eval_results.head()

In [ ]:
# parse out the completion from LLM as a judge and column bind
# the fields of the dictionary to the original results dataframe
df_eval_results_only = df_eval_results['completion'].apply(parse_as_json).apply(pd.Series)
df_eval_results_only.dropna(axis=1, how='all')
df_eval_results = pd.concat([df_eval_results, df_eval_results_only], axis=1)
df_eval_results.rename(columns={'model_id': 'judge_model_id'}, inplace=True)
logger.info(f"df_eval_results shape={df_eval_results.shape}")
df_eval_results.dropna(axis=1, how='all')
df_eval_results.head()

In [ ]:
# send the raw results as a csv file to the S3 bucket
csv_buffer = io.StringIO()
df_eval_results.to_csv(csv_buffer, index=False)
eval_llm_as_a_judge_results = csv_buffer.getvalue()
eval_results_csv_fpath = os.path.join(METRICS_DIR, MODEL_EVAL_COMPLETIONS_CSV)  # Define full S3 path

# Write the CSV data to S3
write_to_s3(eval_llm_as_a_judge_results, BUCKET_NAME, "", 
            METRICS_DIR, MODEL_EVAL_COMPLETIONS_CSV)
logger.info(f"Per PoLL model responses saved as a csv to s3://{BUCKET_NAME}/{eval_results_csv_fpath}")
df_eval_results.head()

In [ ]:
df_eval_results['verdict']

In [ ]:
df_eval_results['verdict'].isin(['incorrect']).any()
df_eval_results[df_eval_results['verdict'] == 'incorrect']

### Send the incorrect and correct responses to S3 separately in `CSV` files for downstream analytics for each model judge
---

In [ ]:
df_incorrect_verdicts = pd.DataFrame()
if df_eval_results['verdict'].isin(['incorrect']).any():
    df_incorrect_verdicts = df_eval_results[df_eval_results['verdict'] == 'incorrect']
    csv_buffer = io.StringIO()
    df_incorrect_verdicts.to_csv(csv_buffer, index=False)
    incorrect_verdict_responses = csv_buffer.getvalue()
    incorrect_verdict_responses_fpath = os.path.join(METRICS_DIR, INCORRECT_VERDICT_RESPONSES_FILE)  # Define full S3 path
    write_to_s3(incorrect_verdict_responses, BUCKET_NAME, "", 
                METRICS_DIR, INCORRECT_VERDICT_RESPONSES_FILE)
    logger.info(f"Incorrect verdict responses from the PoLL evaluation sent to s3://{BUCKET_NAME}/{incorrect_verdict_responses_fpath}")
df_incorrect_verdicts.head()

In [ ]:
logger.info(f"Number of incorrect responses in total: {df_incorrect_verdicts.shape[0]}")

In [ ]:
df_correct_verdicts = pd.DataFrame() 
if df_eval_results['verdict'].isin(['correct']).any():
    # save the correct responses from the model evaluation process in a csv file to analyze where each model went right
    df_correct_verdicts = df_eval_results[df_eval_results['verdict'] == 'correct']
    csv_buffer = io.StringIO()
    df_correct_verdicts.to_csv(csv_buffer, index=False)
    correct_verdict_responses = csv_buffer.getvalue()
    correct_verdict_responses_fpath = os.path.join(METRICS_DIR, CORRECT_VERDICT_RESPONSES_FILE)  # Define full S3 path
    write_to_s3(correct_verdict_responses, BUCKET_NAME, "", 
                METRICS_DIR, CORRECT_VERDICT_RESPONSES_FILE)
    logger.info(f"Correct verdict responses from the PoLL evaluation sent to s3://{BUCKET_NAME}/{correct_verdict_responses_fpath}")
df_correct_verdicts.head()

In [ ]:
logger.info(f"Number of correct responses in total: {df_correct_verdicts.shape[0]}")

In [ ]:
judge_model_ids = df_eval_results['judge_model_id'].unique()
for judge_model_id in judge_model_ids:
    # Filter for incorrect and correct verdicts
    df_judge_incorrect = df_incorrect_verdicts[df_incorrect_verdicts['judge_model_id'] == judge_model_id]
    df_judge_correct = df_correct_verdicts[df_correct_verdicts['judge_model_id'] == judge_model_id]

    # Save incorrect verdicts for each judge model for further analysis to s3 
    if not df_judge_incorrect.empty:
        txt_buffer_incorrect = io.StringIO()
        for index, row in df_judge_incorrect.iterrows():
            txt_buffer_incorrect.write(f"candidate model: {row['candidate_model']}\ncandidate model response: {row['candidate_model_response']}\nground truth: {row['ground_truth']}\nverdict and explanation: {row['completion']}\n\n")
        incorrect_verdict_responses = txt_buffer_incorrect.getvalue()
        incorrect_verdict_responses_fpath = os.path.join(METRICS_DIR, f"{judge_model_id}_incorrect_verdicts_evaluation.txt")
        write_to_s3(incorrect_verdict_responses, BUCKET_NAME, "", METRICS_DIR, f"{judge_model_id}_incorrect_verdicts_evaluation.txt")
        logger.info(f"Incorrect verdict responses for judge {judge_model_id} saved to s3://{BUCKET_NAME}/{incorrect_verdict_responses_fpath}")

    # Save correct verdicts for each judge model for further analysis to s3 
    if not df_judge_correct.empty:
        txt_buffer_correct = io.StringIO()
        for index, row in df_judge_correct.iterrows():
            txt_buffer_correct.write(f"candidate model: {row['candidate_model']}\ncandidate model response: {row['candidate_model_response']}\nground truth: {row['ground_truth']}\nverdict and explanation: {row['completion']}\n\n")
        correct_verdict_responses = txt_buffer_correct.getvalue()
        correct_verdict_responses_fpath = os.path.join(METRICS_DIR, f"{judge_model_id}_correct_verdicts_evaluation.txt")
        write_to_s3(correct_verdict_responses, BUCKET_NAME, "", METRICS_DIR, f"{judge_model_id}_correct_verdicts_evaluation.txt")
        logger.info(f"Correct verdict responses for judge {judge_model_id} saved to s3://{BUCKET_NAME}/{correct_verdict_responses_fpath}")


#### Check for each panel of LLM evaluator's verdict count on the dataset
---

In [ ]:
panel_summary_responses_df = df_eval_results.groupby(['judge_model_id', 'candidate_model', 'verdict']).agg(
    count=('verdict', 'size'),
    mean_cosine_similarity=('cosine_similarity_score', 'mean'),
    mean_levenshtein_distance=('levenshtein_distance', 'mean'),
    mean_token_set_ratio=('token_set_ratio_value', 'mean')
).unstack(fill_value=0).stack().reset_index()

# Save the raw results to a CSV file and upload to S3
csv_buffer = io.StringIO()
panel_summary_responses_df.to_csv(csv_buffer, index=False)
panel_summary_responses = csv_buffer.getvalue()
llm_as_a_judge_per_eval_summary_fpath = os.path.join(METRICS_DIR, LLM_JUDGE_PANEL_RESPONSE_SUMMARIES)

write_to_s3(panel_summary_responses, BUCKET_NAME, "", METRICS_DIR, LLM_JUDGE_PANEL_RESPONSE_SUMMARIES)
logger.info(f"Summary on each eval (max voting/average pooling) for each panel judge sent to s3://{BUCKET_NAME}/{llm_as_a_judge_per_eval_summary_fpath}")
panel_summary_responses_df.head(40)

#### Calculate the overall accuracy of each model scored by the PoLL
---

In [ ]:
per_panel_judgement_result_df = panel_summary_responses_df.pivot_table(
    index=['candidate_model', 'judge_model_id'],
    columns='verdict',
    values='count',
    fill_value=0
).reset_index()

# Ensure 'correct' and 'incorrect' columns exist
if 'correct' not in per_panel_judgement_result_df.columns:
    per_panel_judgement_result_df['correct'] = 0
if 'incorrect' not in per_panel_judgement_result_df.columns:
    per_panel_judgement_result_df['incorrect'] = 0

# Calculate accuracy and error rate
per_panel_judgement_result_df['accuracy'] = (
    per_panel_judgement_result_df.apply(
        lambda row: 100 if row['incorrect'] == 0 else round(row['correct'] / (row['correct'] + row['incorrect']), 2) * 100,
        axis=1
    )
)
per_panel_judgement_result_df['error_rate'] = (
    per_panel_judgement_result_df.apply(
        lambda row: 0 if row['incorrect'] == 0 else round(row['incorrect'] / (row['correct'] + row['incorrect']), 2) * 100,
        axis=1
    )
)

per_panel_judgement_result_df.head(40)

In [ ]:
mean_cosine_similarity = df_eval_results.groupby('candidate_model')['cosine_similarity_score'].mean().reset_index().rename(columns={'cosine_similarity_score': 'mean_cosine_similarity'})
mean_levenshtein_distance = df_eval_results.groupby('candidate_model')['levenshtein_distance'].mean().reset_index().rename(columns={'levenshtein_distance': 'mean_levenshtein_distance'})
mean_token_set_ratio = df_eval_results.groupby('candidate_model')['token_set_ratio_value'].mean().reset_index().rename(columns={'token_set_ratio_value': 'mean_token_set_ratio_value'})
overall_accuracy_grouped_panel_df = per_panel_judgement_result_df.groupby('candidate_model')[['accuracy', 'error_rate']].mean().reset_index()
overall_accuracy_grouped_panel_df = (
    pd.merge(mean_cosine_similarity, overall_accuracy_grouped_panel_df, on='candidate_model')
      .merge(mean_levenshtein_distance, on='candidate_model')
      .merge(mean_token_set_ratio, on='candidate_model')
      .sort_values(by='accuracy', ascending=False)
)
overall_accuracy_grouped_panel_df.head()

In [ ]:
# send the accuracy metrics to s3
csv_buffer = io.StringIO()
overall_accuracy_grouped_panel_df.to_csv(csv_buffer, index=False)
overall_panel_result = csv_buffer.getvalue()
overall_panel_accuracy_metrics_fpath = os.path.join(METRICS_DIR, PER_MODEL_ACCURACY_POLL)  # Define full S3 path

# Write the CSV data to S3
write_to_s3(overall_panel_result, BUCKET_NAME, "", 
            METRICS_DIR, PER_MODEL_ACCURACY_POLL)
logger.info(f"Overall accuracy and error rates results of each model sent to s3://{BUCKET_NAME}/{overall_panel_accuracy_metrics_fpath}")
overall_accuracy_grouped_panel_df.head(10)

In [ ]:
# get accuracy statements
# Rank models by accuracy
ranked_models = overall_accuracy_grouped_panel_df.sort_values(by='accuracy', ascending=False)
highest_accuracy = ranked_models['accuracy'].max()

# Group models with the highest accuracy
top_performers = ranked_models[ranked_models['accuracy'] == highest_accuracy]
other_models = ranked_models[ranked_models['accuracy'] < highest_accuracy]
final_ranking = pd.concat([top_performers, other_models])
unique_judge_model_ids = per_panel_judgement_result_df['judge_model_id'].unique()
PoLL_model_ids = ', '.join(map(str, unique_judge_model_ids))
top_performing_model_ids = ', '.join(top_performers['candidate_model'].tolist())

# cosine similarity score data
highest_cosine_model = final_ranking.loc[final_ranking['mean_cosine_similarity'].idxmax()]
highest_cosine_model_name = highest_cosine_model['candidate_model']
highest_cosine_similarity = highest_cosine_model['mean_cosine_similarity']

In [ ]:
if other_models.empty:
    other_models = f"All models performed the same with an accuracy of {highest_accuracy:.2f}."

In [ ]:
accuracy_statement = MAX_VOTING_RESULT_STATEMENT.format(
    judge_model_ids=PoLL_model_ids,
    highest_accuracy=highest_accuracy,
    top_models=top_performers.to_string(index=False),
    highest_cosine_similarity=round(highest_cosine_similarity, 4),
    top_cosine_similarity_model=highest_cosine_model_name,
    ranked_models=other_models,
    top_performing_model_ids=top_performing_model_ids
)

In [ ]:
# send the overall accuracy report to s3
txt_buffer = io.StringIO()
txt_buffer.write(accuracy_statement)
poll_txt_file_content = txt_buffer.getvalue()
overall_panel_accuracy_metrics_fpath = os.path.join(METRICS_DIR, OVERALL_POLL_REPORT)
write_to_s3(poll_txt_file_content, BUCKET_NAME, "", 
            METRICS_DIR, OVERALL_POLL_REPORT)
logger.info(f"Overall accuracy and error rates results of each model sent to s3://{BUCKET_NAME}/{overall_panel_accuracy_metrics_fpath}")
print(accuracy_statement)

### Final analysis of incorrect responses to spot trends, patterns & observations
---

In this portion of the step, we do the following:

1. We add another layer of a final LLM that goes above all of the verdicts, questions, ground truth, and then gives a set of trends and observations as to how the candidate models performed and what to do to improve their performance.

1. This makes scaling efficient. After getting a quick `vibe check` from the PoLL evaluation results, users can read through the thorough report to get a sense of what to change to make a particular model perform better than the rest and make an efficient evaluation decision/run this evaluation pipeline again.

In [ ]:
# Write all explanations to a file and send to S3
explanations_txt_buffer = io.StringIO()
for index, row in df_eval_results.iterrows():
    explanations_txt_buffer.write(f"candidate model: {row['candidate_model']}\ncandidate model response: {row['candidate_model_response']}\nground truth: {row['ground_truth']}\nverdict and explanation: {row['completion']}\n\n")

explanations_txt_file_content = explanations_txt_buffer.getvalue()
explanations_fpath = os.path.join(METRICS_DIR, ALL_EVALUATIONS_IN_TXT)
write_to_s3(explanations_txt_file_content, BUCKET_NAME, "", METRICS_DIR, ALL_EVALUATIONS_IN_TXT)
logger.info(f"All text eval content from the llm judge panelists sent to s3://{BUCKET_NAME}/{explanations_fpath}")
print(explanations_txt_file_content)

In [ ]:
# open the prompt template and prepare it for inference
with open(os.path.join(eval_prompts_dir, model_eval_subjective_info.get('final_evaluation_prompt_template', None)), 'r') as file:
    final_summary_prompt = file.read()
    processed_summary_eval_prompt: str = final_summary_prompt.format(context=explanations_txt_file_content)

In [ ]:
final_evaluation_summarizer: str = model_eval_subjective_info.get('final_evaluation_summarizer', None)

try:
    logger.info(f"Trying to get a final summary from the inferences, verdicts and explanations")
    final_analysis = completion(
      model=final_evaluation_summarizer,
      messages=[{ "content": processed_summary_eval_prompt,"role": "user"}]
    )

    final_summary_buffer = io.StringIO()
    final_summary_buffer.write(final_analysis['choices'][0]['message']['content'] + "\n")
    final_summary_content = final_summary_buffer.getvalue()
    final_summary_fpath = os.path.join(METRICS_DIR, FINAL_ANALYSIS_REPORT)
    write_to_s3(final_summary_content, BUCKET_NAME, "", METRICS_DIR, FINAL_ANALYSIS_REPORT)
    logger.info(f"Final summary analysis sent to s3://{BUCKET_NAME}/{final_summary_fpath}")
    print(final_summary_content)
except Exception as e:
    logger.error(f"Could not get the final summary analysis on the evaluations: {e}")